## Определение текущего уровня потребительской лояльности

Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и необходимо подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 
Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

In [23]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [25]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [27]:
query = """
    SELECT
      u.user_id,
      u.lt_day,
        CASE
          WHEN u.lt_day <= 365 THEN 'новый'
          WHEN u.lt_day > 365 THEN 'старый'
          ELSE 'unknown'
        END AS is_new,
      u.age,
        CASE
          WHEN u.gender_segment = 1 THEN 'женский'
          WHEN u.gender_segment = 0 THEN 'мужской'
          ELSE 'другой'
        END AS gender,
      u.os_name,
      u.cpe_type_name,
      l.country,
      l.city,
      SUBSTRING(a.title, 3) AS age_segment,
      SUBSTRING(t.title, 3) AS traffic_segment,
      SUBSTRING(ls.title, 3) AS lifetime_segment,
      u.nps_score,
        CASE
          WHEN u.nps_score >=9 THEN 'cторонники'
          WHEN u.nps_score >= 7 THEN 'нейтралы'
          WHEN u.nps_score <= 6 THEN 'критики'
        END AS nps_group
 
    FROM user AS u
    LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
    LEFT OUTER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
    LEFT OUTER JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
    LEFT OUTER JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id;
"""

In [29]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,старый,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,старый,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники


In [30]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Проект: https://public.tableau.com/views/_17415277090900/sheet14?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link